In [6]:
# Change paths according to the correct user. Uncommenct/Comment the desired user.
# Jean-David Therrien
#path_VdQ = "/Users/jeandavidt/OneDrive - Université Laval/COVID/Latest Data/Input/CentrEau-COVID_Resultats_Quebec_final.xlsx"
#images_folder_VdQ = "/Users/jeandavidt/OneDrive - Université Laval/COVID/Website figures"
#path_MTL = "/Users/jeandavidt/OneDrive - Université Laval/COVID/Latest Data/Input/CentrEau-COVID_Resultats_Montreal_final.xlsx"
#images_folder_MTL = "/Users/jeandavidt/OneDrive - Université Laval/COVID/Website figures"

# Niels Nicolai
path_VdQ = "D:/Aplica/One Drive NN/Université Laval/Jean-David Therrien - Latest Data/Input/CentrEau-COVID_Resultats_Quebec_final.xlsx"
images_folder_VdQ = "C:/Users/ANNED2/Downloads"
path_MTL = "D:/Aplica/One Drive NN/Université Laval/Jean-David Therrien - Latest Data/Input/CentrEau-COVID_Resultats_Montreal_final.xlsx"
images_folder_MTL = "C:/Users/ANNED2/Downloads"
import sys; sys.path.insert(0,'C:/Users/ANNED2/AppData/Local/Continuum/anaconda3/envs/covid/Library/bin') #Adds correct path for the kaleido.cmd file


In [9]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import requests
import json
from plotly.colors import DEFAULT_PLOTLY_COLORS
import plotly.express as px

In [10]:

def get_cases_from_ledevoir(region_name, start_date=None, end_date=None):
    response = requests.get("https://ledevoir-coronavirus.herokuapp.com/api/v2/reports/ca/qc")
    j = json.loads(response.text)
    region_codes = {j["regions"][i]["name"]: i for i in range(len(j["regions"]))}
    region_code = region_codes[region_name]
    cases = pd.DataFrame(j["regions"][region_code]["data"])
    cases["date"] = pd.to_datetime(cases["date"])
    cases = cases.set_index("date")
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    if pd.isna(start_date) and pd.isna(end_date):
        pass
    elif pd.isna(end_date):
        cases = cases.loc[start_date:]
    elif pd.isna(start_date):
        cases = cases.loc[:end_date]
    else:
        cases = cases.loc[start_date : end_date]
    return cases


NORM_COL = "SARS/PMMV (gc/gc)"
SARS_COL = "SARS (gc/ml)"
PMMV_COL = "PMMV (gc/ml)"


def plot(data, what, proper_names, region_name, proper_region_name, max_value=None, start_date=None, end_date=None):
    var_map = {
        "sars": SARS_COL,
        "normalized": NORM_COL,
        "pmmv": PMMV_COL,
    }
    if what not in var_map.keys():
        raise KeyError("Don't know what to plot: " + what)
    cols = []
    for i, _ in enumerate(proper_names):
        new_col = var_map[what] + "." + str(i)
        new_col = new_col.replace(".0", "")
        cols.append(new_col)
    fig = make_subplots(rows=1, cols=1,
                    specs=[[{"secondary_y": True}]])
    traces = []
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    if pd.isna(start_date) and pd.isna(end_date):
        pass
    elif pd.isna(end_date):
        data = data.loc[data["Date"] > start_date]
    elif pd.isna(start_date):
        data = data.loc[data["Date"] < end_date]
    else:
        data = data.loc[(data["Date"] > start_date) & (data["Date"] < end_date)]
    data = data.dropna(subset = cols, how="all")
    colors = px.colors.qualitative.Plotly
    line_colors = [color for i, color in enumerate(colors) if i != 2]
    bar_color = colors[2]
    for i, (col, name) in enumerate(zip(cols, proper_names)):
        trace = go.Scatter(
            x=data["Date"],
            y=data[col],
            name=name,
            mode="lines+markers",
            marker=dict(color=line_colors[i%len(line_colors)]),
            connectgaps=True,

        )
        traces.append(trace)
    for trace in traces:
        fig.add_trace(trace, secondary_y=True)
    fig.update_layout(
        xaxis_title="Date",
        xaxis_tick0="2020-12-27",
        xaxis_dtick=7 * 24 * 3600000,
        xaxis_tickformat="%d-%m-%Y",
        xaxis_tickangle=30, plot_bgcolor="white",
        xaxis_gridcolor="rgba(100,100,100,0.10)",
        yaxis_gridcolor="rgba(0,0,0,0)",
        xaxis_ticks="outside"
    )
    axis_titles = {
        "sars": "Copies du SRAS-CoV-2/ml",
        "pmmv": "Copies du PMMoV/ml",
        "normalized": "SRAS-CoV-2/PMMoV",
    }
    fig.update_yaxes(title=axis_titles[what], secondary_y=True, range=[0, max_value])

    fig.update_layout(title=dict(text=f"Surveillance SRAS-CoV-2 via les eaux usées<br>{proper_region_name}"))

    cases = get_cases_from_ledevoir(region_name, start_date=start_date, end_date =end_date)
    cases_trace = go.Bar(x=cases.index, y=cases["dc"], name="Nouveaux cas<br>journaliers", marker=dict(opacity=0.3, color=bar_color))

    fig.add_trace(cases_trace, secondary_y=False)
    fig.update_layout(legend=dict(yanchor="top", xanchor="left", orientation="h", y=1.1, x=0))
    fig.update_yaxes(title="Nouveaux cas",side="right", secondary_y=False)
    fig.update_yaxes(side="left", secondary_y=True)
    fig.add_layout_image(
    dict(
        source="https://www.centreau.ulaval.ca/fileadmin/Documents/Image_de_marque/102378_MODIF_LOGO-CENTREAU_noir.jpg",
        xref="paper", yref="paper",
        x=1, y=1.00,
        sizex=0.25, sizey=0.25,
        xanchor="right", yanchor="bottom"
        )
    )
    return fig

def remove_points(df, points_to_remove):
    base_names = [NORM_COL, SARS_COL, PMMV_COL]
    for _, info in points_to_remove.items():
        idx = str(info["index"])
        col_names = [base + f".{idx}" for base in base_names]
        clean_names = []
        for col_name in col_names:
            col_name = col_name.replace(".0", "")
            clean_names.append(col_name)
        df.loc[df["Date"].isin(info["dates"]), clean_names] = np.nan
    return df

# Normalized Figures

In [11]:
path = path_VdQ
qc = pd.read_excel(path, sheet_name="QC_Compil_STEP (#NA)", usecols="A:N", skiprows=4)
qc = qc.rename(columns={"Data":"Date"})

start_date = "2021-01-01"
end_date = None

SITES = ["Québec Station Est", "Québec Station Ouest"]  # in the order that they are found in the excel sheet

points_to_remove = {
    "Québec Station Est": {
        "index": 0,
        "dates": [
            "2021-04-09",
        ],
    },
    "Québec Station Ouest": {
        "index": 1,
        "dates": [
            "2021-03-24",
            "2021-04-03",
            "2021-04-11",
            "2021-04-12",
            "2021-05-01",
        ],
    },
}
qc = remove_points(qc, points_to_remove)


region_name = "Capitale-Nationale"
proper_region_name = "Région de Québec"

# 'what' should either be "pmmv", "sars" or "normalized"
fig = plot(qc, "normalized", SITES, region_name, proper_region_name, 
    max_value=None, start_date=start_date, end_date=end_date)
fig.update_yaxes(secondary_y=False, range=[0, 550])
images_folder = images_folder_VdQ
fig.write_image(os.path.join(images_folder, "Quebec.png"), width=1000, scale=3)
fig.write_image(os.path.join(images_folder, "Quebec.svg"), width=1000)
fig.write_html(images_folder+"test_plot.html")
fig.show()

C:\Users\ANNED2\AppData\Local\Continuum\anaconda3\envs\covid\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


## Normalized 2021 For Montreal

In [14]:
path = path_MTL
mtl_step = pd.read_excel(path, sheet_name="MTL_Compil_STEP (#NA)", usecols="A:N", skiprows=4)
mtl_step = mtl_step.rename(columns={"Data":"Date"})
mtl_step["Date"] = pd.to_datetime(mtl_step["Date"])

start_date = "2021-01-01"
end_date = None

norm_cols = ["SARS/PMMV (gc/gc)", "SARS/PMMV (gc/gc).1"]
proper_names = ["Montréal<br>Intercepteur Nord", "Montréal<br>Intercepteur Sud"]
region_name = "Montréal"
proper_region_name = "Montréal"

fig = plot(mtl_step, "normalized", proper_names, region_name, proper_region_name, max_value=None, start_date=start_date, end_date=end_date)

fig.update_yaxes(secondary_y=False, range=[0, 1250])
images_folder = images_folder_MTL
fig.write_image(os.path.join(images_folder, "Montreal-2021.png"), width=1000, scale=3)
fig.write_image(os.path.join(images_folder, "Montreal-2021.svg"), width=1000)
fig.show()

C:\Users\ANNED2\AppData\Local\Continuum\anaconda3\envs\covid\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning:

Data Validation extension is not supported and will be removed



In [15]:
path = path_MTL
mtl_step = pd.read_excel(path, sheet_name="MTL_Compil_STEP (#NA)", usecols="A:N", skiprows=4)
mtl_step = mtl_step.rename(columns={"Data":"Date"})
mtl_step["Date"] = pd.to_datetime(mtl_step["Date"])

start_date = "29-02-2020"
end_date = None

norm_cols = ["SARS/PMMV (gc/gc)", "SARS/PMMV (gc/gc).1"]
proper_names = ["Montréal<br>Intercepteur Nord", "Montréal<br>Intercepteur Sud"]
region_name = "Montréal"
proper_region_name = "Montréal"

fig = plot(mtl_step, "normalized", proper_names, region_name, proper_region_name, max_value=1.35, start_date=start_date, end_date=end_date)
fig.update_layout(
        xaxis_title="Date",
        xaxis_dtick="M1",
        xaxis_tick0="2020-01-01",
        xaxis_tickformat="%d-%m-%Y",
    )
fig.update_yaxes(secondary_y=False, range=[0, 1250])
images_folder = images_folder_MTL
fig.write_image(os.path.join(images_folder, "Montreal.png"), width=1000, scale=3)
fig.write_image(os.path.join(images_folder, "Montreal.svg"), width=1000)
fig.show()

C:\Users\ANNED2\AppData\Local\Continuum\anaconda3\envs\covid\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning:

Data Validation extension is not supported and will be removed



In [6]:
path = path_MTL
mtl_reseau = pd.read_excel(path, sheet_name="MTL_Compil_RESEAU (Blank)", usecols="A:CS", skiprows=4)
mtl_reseau = mtl_reseau.rename(columns={"Data":"Date"})
mtl_reseau["Date"] = pd.to_datetime(mtl_reseau["Date"])

start_date = "2021-02-25"
end_date = "2021-04-09"

norm_cols = ["SARS/PMMV (gc/gc)", "SARS/PMMV (gc/gc).3", "SARS/PMMV (gc/gc).6", "SARS/PMMV (gc/gc).9", "SARS/PMMV (gc/gc).12", "SARS/PMMV (gc/gc).15"]
proper_names = ["Ville St-Laurent", "Côte-Des-Neiges", "Prison de Bordeaux", "Ville d'Anjou", "Montréal-Nord Secteur Est-1", "Montréal-Nord Secteur Est-2"]
region_name = "Montréal"
proper_region_name = "Réseau d'égoûts de Montréal"
max_normalized = 1.25
fig = plot(mtl_reseau, norm_cols, "normalized", proper_names, region_name, proper_region_name, max_normalized, start_date=start_date, end_date=end_date)
images_folder = images_folder_MTL
fig.write_image(os.path.join(images_folder, "mtl_reseau_cpTP.png"), width=1000, scale=3)
fig.show()
fig.write_html(os.path.join(images_folder, "test_fig.html"))

TypeError: plot() got multiple values for argument 'start_date'

# SARS only

In [10]:
path = path_VdQ

qc = pd.read_excel(path, sheet_name="QC_Compil_STEP (Blank)", usecols="A:N", skiprows=4)
qc = qc.rename(columns={"Data":"Date"})

start_date = "2021-01-01"
end_date = "2021-04-13"

cols = ["SARS (gc/ml)", "SARS (gc/ml).1"]
# sars_cols = ["SARS (gc/ml)", "SARS (gc/ml).1"]
proper_names = ["Québec Station Est", "Québec Station Ouest"]
region_name = "Capitale-Nationale"
proper_region_name = "Région de Québec"
max_val = None
fig = plot(qc, cols, "normalized", proper_names, region_name, proper_region_name, 
    max_val, start_date=start_date, end_date=end_date)
fig.update_yaxes(secondary_y=False, range=[0, 550])
images_folder = images_folder_VdQ
fig.write_image(os.path.join(images_folder, "Quebec-sars.png"), width=1000, scale=3)
fig.write_image(os.path.join(images_folder, "Quebec-sars.svg"), width=1000)
fig.write_html(os.path.join(images_folder, "test_plot.html"))
fig.show()

/usr/local/Caskroom/miniconda/base/envs/covid/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning:

Data Validation extension is not supported and will be removed



In [6]:
path = path_MTL
mtl_step = pd.read_excel(path, sheet_name="MTL_Compil_STEP (#NA)", usecols="A:N", skiprows=4)
mtl_step = mtl_step.rename(columns={"Data":"Date"})
mtl_step["Date"] = pd.to_datetime(mtl_step["Date"])

start_date = "29-02-2020"
end_date = "2021-04-13"

norm_cols = ["SARS (gc/ml)", "SARS (gc/ml).1"]
proper_names = ["Montréal<br>Intercepteur Nord", "Montréal<br>Intercepteur Sud"]
region_name = "Montréal"
proper_region_name = "Montréal"
max_normalized = None
fig = plot(mtl_step, norm_cols, "sars", proper_names, region_name, proper_region_name, max_normalized, start_date=start_date, end_date=end_date)
fig.update_layout(
        xaxis_title="Date",
        xaxis_dtick="M1",
        xaxis_tick0="2020-01-01",
        xaxis_tickformat="%d-%m-%Y",
    )
fig.update_yaxes(secondary_y=False, range=[0, 1250])
images_folder = images_folder_MTL
fig.write_image(os.path.join(images_folder, "Montreal-sars.png"), width=1000, scale=3)
fig.write_image(os.path.join(images_folder, "Montreal-sars.svg"), width=1000)
fig.show()

/usr/local/Caskroom/miniconda/base/envs/covid/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning:

Data Validation extension is not supported and will be removed

